In [1]:
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import random

Using TensorFlow backend.


In [6]:
data1='D:\\fintest\\1coal'
data2='D:\\fintest\\2coal'
data3='D:\\fintest\\3coal'


def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,0,1])])
        
    shuffle(images)
    return images

   

data = data_with_label()

random.shuffle(data)
train_data, test_data = train_test_split(data, test_size=0.2)

from keras.applications.resnet50 import preprocess_input
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1,224,224,3)
tr_img_data = preprocess_input(tr_img_data)
tr_lbl_data = np.array([i[1] for i in train_data])
tst_img_data = np.array([i[0] for i in test_data]).reshape(-1,224,224,3)
tst_img_data = preprocess_input(tst_img_data)
tst_lbl_data = np.array([i[1] for i in test_data])



100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [00:23<00:00, 20.47it/s]


In [9]:
def create_model():
    
    
    from keras.applications.resnet50 import ResNet50
    from keras.models import Model
    import keras
    restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    output = restnet.layers[-1].output
    output = keras.layers.Flatten()(output)
    restnet = Model(restnet.input, output=output)
    for layer in restnet.layers:
        layer.trainable = False
    import pandas as pd
    restnet.trainable = True
    set_trainable = False
    for layer in restnet.layers:
        if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    layers = [(layer, layer.name, layer.trainable) for layer in restnet.layers]
    from keras.models import Sequential
    from keras import optimizers
    model_finetuned = Sequential()
    model_finetuned.add(restnet)
    model_finetuned.add(Dense(512, activation='relu', input_dim=(224,224,1)))
    model_finetuned.add(Dropout(0.5))
    model_finetuned.add(Dense(3, activation='sigmoid'))
    model_finetuned.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-5),
                  metrics=['accuracy'])
    return model_finetuned

In [10]:
checkpoint_path = "cpresnet.ckpt"
model=create_model()

model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(tst_img_data,  tst_lbl_data, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

C:\Users\Abhilasha\Anaconda2\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  # Remove the CWD from sys.path while we load stuff.


Restored model, accuracy: 87.34%


In [13]:
for img in tst_img_data:

    img =img.reshape(-1,224,224,3)
    model_out=model.predict([img])
    print(model_out)
    if np.argmax(model_out)==0:
        str_label='1'
        
    elif np.argmax(model_out)==1:
        str_label='2'
    else:
        str_label='3'
    print(str_label)
    

[[6.1068835e-04 2.3679115e-06 4.8156159e-07]]
1
[[6.6465713e-02 2.7499247e-08 2.6331790e-10]]
1
[[1.9172133e-12 6.2915251e-10 1.3030227e-04]]
3
[[7.7493176e-09 6.7433796e-04 2.9888254e-04]]
2
[[7.5917833e-02 2.5477905e-06 1.0006088e-07]]
1
[[2.5621630e-10 5.4833103e-09 7.4688091e-05]]
3
[[1.8475791e-06 1.5733216e-09 2.1189415e-09]]
1
[[1.1184352e-06 2.2552702e-09 4.8913851e-10]]
1
[[9.7479855e-09 8.3258790e-05 1.4647404e-02]]
3
[[3.0653151e-03 3.3885218e-08 3.6822947e-10]]
1
[[2.2881583e-04 2.0410609e-09 1.0360021e-09]]
1
[[6.4358391e-02 6.0742846e-06 1.5877094e-10]]
1
[[3.2249366e-12 2.0155175e-08 1.3788996e-03]]
3
[[4.3058212e-06 5.5074011e-06 1.2911644e-05]]
3
[[4.9502785e-05 1.6325728e-07 1.0174600e-11]]
1
[[6.7589822e-04 1.6816308e-07 3.2563364e-08]]
1
[[8.3511434e-11 1.1017594e-08 4.7407343e-04]]
3
[[1.6316579e-05 4.5412849e-05 2.5721461e-05]]
2
[[1.5231562e-10 1.3853625e-08 2.3281744e-03]]
3
[[3.9696999e-07 6.6049850e-08 1.2900085e-04]]
3
[[1.5189111e-06 2.7216482e-05 3.1511542e

[[6.2280446e-02 2.5535012e-08 1.3242990e-06]]
1
[[3.1123310e-08 1.9400136e-07 1.2551082e-04]]
3
[[5.0175939e-09 1.0167461e-07 1.1596555e-07]]
3
[[1.4145364e-07 2.6656082e-08 4.4744426e-09]]
1
[[3.2720172e-07 2.3321709e-08 1.4368731e-10]]
1
[[1.3400498e-06 3.6723717e-04 4.3718563e-05]]
2
[[1.7352148e-04 3.2859964e-06 9.1431633e-05]]
1
[[1.3633498e-10 3.9999611e-07 9.0141926e-04]]
3
[[1.9918689e-06 1.0589638e-04 2.8973154e-04]]
3
[[2.5621630e-10 5.4833103e-09 7.4688091e-05]]
3
[[6.1995036e-07 2.2768359e-06 1.9121248e-06]]
2
[[3.275615e-04 1.686185e-06 4.380440e-03]]
3
[[4.133655e-09 5.742997e-09 8.702915e-04]]
3
[[4.5499071e-10 8.4739797e-09 1.4134934e-04]]
3
[[5.9408936e-09 6.1545741e-08 5.3018419e-05]]
3
[[3.3504995e-05 2.2262767e-04 1.5129561e-07]]
2
[[5.6596823e-06 8.3637612e-09 5.1235418e-07]]
1
[[9.8918395e-07 6.8935783e-06 3.0002673e-05]]
3
[[2.367731e-06 2.823206e-04 9.438166e-04]]
3
[[5.0694744e-06 4.5313923e-06 3.0322832e-03]]
3
[[1.4386956e-08 2.7179998e-09 8.0422957e-08]]
3
[